In [184]:
import numpy as np
import pandas as pd
import random
 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torch.utils.data import DataLoader,TensorDataset
from sklearn.preprocessing import MinMaxScaler

In [185]:
sequence_length = 10 # 원하는 sequence_length 설정
batch_size = 36 # 원하는 batch_size 설정 >> 6시간 기준
hidden_size = 16 # 원하는 hidden_size 설정
input_size = 11 # 사용하는 feature의 수
output_size = 1

In [186]:
# df = pd.read_csv('../../data/Rail_data.csv')
df = pd.read_csv('../../data/Rail_data_split.csv')
df1 = pd.read_csv('../../data/Rail_data_back_split.csv')
# scaler = MinMaxScaler()
# scaled_col = ['air_temp','TSI','azimuth','altitude','solar_rad','High_solar_rad', 'casi', 'humidity', 'rain', 'wind_speed','wind_direction','rail_direction']
# df[scaled_col]= scaler.fit_transform(df[scaled_col])

# scaler1 = MinMaxScaler()
# df['rail_temp'] = scaler1.fit_transform(df['rail_temp'].values.reshape(-1,1))

df = df.astype({'solar_rad': 'float64'})
df = df.astype({'High_solar_rad': 'float64'})
df = df.astype({'casi': 'float64'})
df = df.astype({'humidity': 'float64'})
df = df.astype({'wind_speed': 'float64'})
df = df.drop(['rail_direction'], axis=1)
# int type >> float type

df1 = df1.astype({'solar_rad': 'float64'})
df1 = df1.astype({'High_solar_rad': 'float64'})
df1 = df1.astype({'casi': 'float64'})
df1 = df1.astype({'humidity': 'float64'})
df1 = df1.astype({'wind_speed': 'float64'})
df1 = df1.drop(['rail_direction'], axis=1)

X = df.iloc[:,:11].values
y = df.iloc[:,11].values

X1 = df1.iloc[:,:11].values
y1 = df1.iloc[:,11].values

def sequence_data(X,y, sequence_size): # 원하는 sequence에 따라 데이터 분리
    x_seq = []
    y_seq = []
    for idx in range(len(X) - sequence_size): #len(X)가 7000이고 seq_size가 5라면?
        x_seq.append(X[idx:idx + sequence_size]) # sequence_lengh개씩 특성들을 모두 묶음 >> shape: 5, 11
        y_seq.append(y[idx + sequence_size])     # x에 따른 온도들을 묶음 >> shape: 5, 1
        
    return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32).view(-1,1)

X_seq, y_seq = sequence_data(X, y, sequence_length)


X_seq, y_seq = sequence_data(X, y, sequence_length) # 원하는 sequence_length에 따라 데이터 묶기
X1_seq, y1_seq = sequence_data(X1, y1, sequence_length)


X_train, X_test = X_seq[:int(len(X_seq)*0.7)], X_seq[int(len(X_seq)*0.7):]
y_train, y_test = y_seq[:int(len(y_seq)*0.7)], y_seq[int(len(y_seq)*0.7):]

X1_train, X1_test = X1_seq[:int(len(X1_seq)*0.7)], X1_seq[int(len(X1_seq)*0.7):]
y1_train, y1_test = y1_seq[:int(len(y1_seq)*0.7)], y1_seq[int(len(y1_seq)*0.7):]

# 0.6, 0.2, 0.2씩 train, val, test로 분리하기

X_train_cat = torch.cat([X_train, X1_train], dim = 0)
y_train_cat = torch.cat([y_train, y1_train], dim = 0)
X_test_cat = torch.cat([X_test, X1_test], dim = 0)
y_test_cat = torch.cat([y_test, y1_test], dim = 0)


train_DS = TensorDataset(X_train_cat, y_train_cat)
test_DS = TensorDataset(X_test_cat, y_test_cat)


train_DL = DataLoader(train_DS, batch_size = batch_size)
test_DL = DataLoader(test_DS, batch_size = batch_size)


# 0.6, 0.2, 0.2씩 train, val, test로 분리하기


# batch_size에 따라 데이터 처리

In [187]:
data = next(iter(train_DL))
print(data[0].shape)
print(data[1].shape)

torch.Size([36, 10, 11])
torch.Size([36, 1])


In [188]:
# dimension = 64
# to_query = nn.Linear(dimension,dimension)
# to_key = nn.Linear(dimension, dimension)
# to_value = nn.Linear(dimension, dimension)

# query = to_query(X_train)
# key = to_key(X_train)
# value = to_value(X_train)

# print(f'Query : {query.shape}')
# print(f'Key : {key.shape}')
# print(f'Value : {value.shape}')

In [189]:
# attention_score = query @ key.permute(0, 1, 3, 2)
# attention_score = attention_score.softmax(dim = -1)

# contextualized_tokens = attention_score @ value

In [198]:

class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, device):
        super(LSTMEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.device = device
        
        self.LSTM = nn.LSTM(input_size, hidden_size, batch_first = True).to(self.device)
        
    def forward(self, input):
        output, hidden = self.LSTM(input) # input: 36, 10, 16/ output: 36, 10, 16 / hidden: 1, 36, 16
        
        return output, hidden 
    
        
class LSTMDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, device):
        super(LSTMDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.device = device
        
        self.LSTM = nn.LSTM(hidden_size, hidden_size, batch_first = True).to(self.device)
        self.output_linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output, hidden = self.LSTM(input, hidden) 
        output = self.output_linear(output) # output: 36, 10, 1 / hidden: 1, 36, 16
        return output, hidden
        
class Attention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device):
        super(Attention, self).__init__()
        
        self.device = device
        self.encoder = LSTMEncoder(input_size, hidden_size, device).to(self.device)
        self.decoder = LSTMDecoder(hidden_size, output_size, device).to(self.device)
        
        self.attention = nn.Linear(hidden_size + output_size, sequence_length).to(self.device)
        
    def forward(self, input):
        input = input.view(-1, sequence_length, input_size) # input: 36, 10, 16
        encoder_output, encoder_hidden = self.encoder(input) # output: 36, 10, 16 / hidden: 1, 36, 16
        
        hidden = encoder_hidden
        outputs = []
        
        for i in range(sequence_length):
            decoder_input = encoder_output[:, i:i+1, :] # 36, 1, 16
            decoder_output, hidden = self.decoder(decoder_input, hidden) 
            # output: 36, 1, 1 / hidden: 1, 36, 16
            
            attn_input = torch.cat((decoder_output, encoder_output[:,i:i+1,:]), dim=2) # input : 36, 1, 17
            attn_weights = torch.softmax(self.attention(attn_input), dim=2) # 36, 1, 10
            context = torch.bmm(attn_weights, encoder_output) # 36, 1, 10 @ 36, 10, 16
            # 36, 1, 16
            decoder_output = torch.cat((decoder_output, context), dim=2)
            outputs.append(decoder_output)
            
        outputs = torch.cat(outputs, dim=1)
        return outputs


In [199]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #gpu 활성화 확인
model = Attention(input_size, hidden_size, output_size, device).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

loss_graph = []
n = len(train_DL)

for epoch in range(100):
    running_loss = 0
    for data in train_DL:
        seq, target = data[0].to(device), data[1].to(device)
        
        out = model(seq)
        loss = criterion(out, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    loss_graph.append(running_loss/n)
    if epoch % 20 == 0:
        print("[epoch: %d] loss : %.4f" %(epoch,running_loss/n))



c:\Users\AML2\Desktop\TIL\venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([36, 1])) that is different to the input size (torch.Size([36, 10, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (10) must match the size of tensor b (36) at non-singleton dimension 1